# 1. Tạo và test Database

```sql
-- Create the database table
CREATE TABLE IF NOT EXISTS video_transcripts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_url TEXT NOT NULL,
    transcription TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Create index for faster URL lookups
CREATE INDEX IF NOT EXISTS idx_video_url ON video_transcripts(video_url);

-- Example of inserting a record
INSERT INTO video_transcripts (video_url, transcription) 
VALUES (
    'https://drive.google.com/file/d/10qe6DkNX6up7-oG69HRG3B4j1A-WiI_o/view',
    'Your video transcription text here'
);
```

# 2. Trích transcriptions từ database ứng URL được cho trước -> sau đó chạy qua API POSTMAN 

```bash
curl --location 'http://localhost:5000/analyze' \
--header 'Content-Type: application/json' \
--data '{"transcription": "[00:04:15] Mentee: Oh, good afternoon.\n[00:04:32] Mentor: Hello.\n[00:04:35] Mentor: Good afternoon."}'
```

```python
# trích transcriptions từ database

# Cho 1 URL_video = https://drive.google.com/file/d/10qe6DkNX6up7-oG69HRG3B4j1A-WiI_o/view

# trích transcriptions ứng với URL_video từ database

import sqlite3

# Kết nối đến database
conn = sqlite3.connect('video_database.db')
cursor = conn.cursor()

# Truy vấn tất cả dữ liệu trong bảng
cursor.execute('SELECT * FROM video_transcripts')

curl --location 'http://localhost:5000/analyze' \

--header 'Content-Type: application/json' \

--data '{"transcription": "[00:04:15] Mentee: Oh, good afternoon.\n[00:04:32] Mentor: Hello.\n[00:04:35] Mentor: Good afternoon."}'

# Lấy tất cả các dòng dữ liệu
rows = cursor.fetchall()

# In ra từng dòng dữ liệu
print("Tổng số bản ghi:", len(rows))
print("\nChi tiết các bản ghi:")
for row in rows:
    print("\nID:", row[0])
    print("Video URL:", row[1])
    print("Transcription:", row[2])
    print("Created at:", row[3])
    print("Updated at:", row[4])
    print("-" * 50)

# Đóng kết nối
conn.close()
```

Code của bạn cần thêm một số điều chỉnh:

1. Thêm trường criteria vào database nếu chưa có
2. Xử lý lỗi chi tiết hơn
3. Thêm kiểm tra response status code
4. Hiển thị kết quả rõ ràng hơn

Đây là code đã sửa:

```python
import sqlite3
import requests
import json

# URL video cụ thể
url_video = "https://drive.google.com/file/d/10qe6DkNX6up7-oG69HRG3B4j1A-WiI_o/view"

# Kết nối đến database
conn = sqlite3.connect('video_database.db')
cursor = conn.cursor()

try:
    # Thêm column criteria nếu chưa tồn tại
    cursor.execute('''
        SELECT COUNT(*) FROM pragma_table_info('video_transcripts') 
        WHERE name='criteria'
    ''')
    if cursor.fetchone()[0] == 0:
        cursor.execute('''
            ALTER TABLE video_transcripts
            ADD COLUMN criteria TEXT
        ''')
        print("Đã thêm cột criteria vào database")

    # Lấy transcription từ database
    cursor.execute('SELECT transcription FROM video_transcripts WHERE video_url = ?', (url_video,))
    result = cursor.fetchone()

    if result:
        transcription = result[0]
        
        # Gửi request đến API
        api_url = 'http://localhost:5000/analyze'
        headers = {'Content-Type': 'application/json'}
        data = {"transcription": transcription}
        
        try:
            response = requests.post(api_url, headers=headers, json=data)
            
            # Kiểm tra status code
            response.raise_for_status()
            
            criteria_json = response.json()
            
            # Lưu kết quả JSON vào database
            cursor.execute('''
                UPDATE video_transcripts 
                SET criteria = ?,
                    updated_at = CURRENT_TIMESTAMP
                WHERE video_url = ?
            ''', (json.dumps(criteria_json), url_video))
            
            conn.commit()
            print("\nĐã cập nhật criteria thành công!")
            
            # Kiểm tra kết quả
            cursor.execute('''
                SELECT id, video_url, transcription, criteria, created_at, updated_at 
                FROM video_transcripts 
                WHERE video_url = ?
            ''', (url_video,))
            updated_result = cursor.fetchone()
            
            if updated_result:
                print("\nThông tin sau khi cập nhật:")
                print("ID:", updated_result[0])
                print("URL:", updated_result[1])
                print("\nTranscription:", updated_result[2][:200] + "...") # Chỉ hiển thị 200 ký tự đầu
                print("\nCriteria:", json.loads(updated_result[3]))
                print("\nCreated at:", updated_result[4])
                print("Updated at:", updated_result[5])
                
        except requests.exceptions.RequestException as e:
            print("Lỗi khi gọi API:", str(e))
        except json.JSONDecodeError as e:
            print("Lỗi khi xử lý JSON:", str(e))
        except Exception as e:
            print("Lỗi không xác định:", str(e))
    else:
        print("Không tìm thấy transcription cho URL này")

except sqlite3.Error as e:
    print("Lỗi database:", str(e))
    
finally:
    # Đóng kết nối
    conn.close()
```

Những thay đổi chính:

1. Thêm kiểm tra và tạo cột criteria nếu chưa tồn tại
2. Thêm try-except để xử lý lỗi database
3. Kiểm tra response.raise_for_status() để phát hiện lỗi HTTP
4. Hiển thị thông tin đầy đủ hơn sau khi cập nhật
5. Giới hạn hiển thị transcription để dễ đọc
6. Thêm finally để đảm bảo đóng kết nối database
7. Thêm chi tiết thông báo lỗi

Code này sẽ hoạt động ổn định hơn và dễ debug nếu có lỗi xảy ra.